In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws,col
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [ ]:
spark = SparkSession \
    .builder \
    .appName("pgML") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Load books data from MongoDB and Create features/label

In [ ]:
database = 'your database'
collection = 'your collection'
user_name = 'your user name'
password = 'your password'
address = 'your address'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [ ]:
pg = spark.read.format("mongo").option("uri",connection_string).load()

In [ ]:
df=pg

### Combine all books features into one text column

In [ ]:
from pyspark.sql.functions import concat_ws,col
df=df.select(df.title,df.aboutTheBook,df.author,df.authorBio,df.categories,df.keynote,df.praises,\
            concat_ws(',',df.title,df.aboutTheBook,df.author,df.authorBio,df.categories,df.keynote,df.praises)
              .alias("text"))
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 title| aboutTheBook| author| authorBio| categories| keynote| praises| text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 The Sensuous Woman|<b>The book that ...| "J"|<b>J </b>is the p...|[Family / Child C...| null|[&quot;[She] is u...|The Sensuous Woma...|
 Kosher Gourmet|Embark on a new a...|92nd Street Y Coo...| null|[Cookbooks & Cook...| null|[&quot;The perfec...|Kosher Gourmet,Em...|
Borreguita and th...|What's a little l...| Verna Aardema|* Verna Aardema g...|[Juvenile, Animal...| null| []|Borreguita and th...|
 Inviting God In|This warm, inspir...| Rabbi David Aaron|Rabbi David Aaron...|[Nonfiction, Reli...|<B>With warmth an...|[&#8220;Aaron&#82...|Inviting God In,T...|
The God-Powered Life|You are an indivi...| Rabbi David Aaron|Rabbi David Aaron...|[Nonfiction, Reli...|<B>A popular rabb...|[&ldquo;Aaron&rsq...|The God-Powered L...|
The Secret Life o...|How much does our...| Rabbi David Aaron|Rabbi David Aaron...|[Nonfiction, Reli...| null|[&quot;With a ton...|The Secret Life o...|
 Catch Me If You Can|<b>The uproarious...|Frank Abagnale, J...|A former professi...|[True Crime, Biog...| null|["A book that cap...|Catch Me If You C...|
 Stealing Your Life|<b>The charismati...| Frank W Abagnale|FRANK W. ABAGNALE...|[Nonfiction, Refe...|Now in paperback,...|[&#8220;Offering ...|Stealing Your Lif...|
The Art of the Steal|The world--famous...| Frank W. Abagnale|A former professi...|[Business / Econo...| null| []|The Art of the St...|
 Grimpow|GRIMPOW HAD NO id...| Rafael Ábalos|<b>Rafael &#193;b...|[Teen & Young Adu...| null|[<b>Starred Revie...|Grimpow,GRIMPOW H...|
 Gardening Eden|<i>Before the sna...| Michael Abbate|A nationally reco...|[Nonfiction, Reli...|People of faith a...|[&#8220;Firmly on...|Gardening Eden,<i...|
 Desert Solitaire|<b>&ldquo;Rough, ...| Edward Abbey|<b>Edward Abbey</...|[Nature Writing, ...| null|[&ldquo;This book...|Desert Solitaire,...|
 Planeswalker|Urza Triumphant...| Lynn Abbey|Lynn Abbey, autho...|[Science Fiction ...| null| []|Planeswalker,Urza...|
 Flatland|&#160;&#160;&#160...| Edwin A. Abbott|Edwin A. Abbott (...|[Fiction / Litera...| null|[&ldquo;At once a...|Flatland,&#160;&#...|
 The Bridesmaid|After years of wa...| Hailey Abbott|Hailey Abbott has...|[Teen & Young Adu...| null| []|The Bridesmaid,Af...|
In the Belly of t...|A visionary book ...| Jack Henry Abbott|Born in Michigan ...|[Institutions and...| null| []|In the Belly of t...|
 Distant Blood|Blood runs cold--...| Jeff V. Abbott|<b>Jeff Abbott</b...|[Mystery, Crime M...| null| []|Distant Blood,Blo...|
 Do Unto Others|<b>&ldquo;One of ...| Jeff Abbott|Jeff Abbott is th...|[Crime Mysteries,...| null|[&ldquo;One of th...|Do Unto Others,<b...|
 Only Good Yankee|Mirabeau, Texas, ...| Jeff Abbott|<b>Jeff Abbott</b...|[Crime Mysteries,...| null|[&quot;A carefull...|Only Good Yankee,...|
 Promises of Home|In a small Texas ...| Jeff V. Abbott|<b>Jeff Abbott</b...|[Mystery, Crime M...| null| []|Promises of Home,...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

### Tokenization on text and compute the TF-IDF features

In [ ]:

# Define tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")

# Apply tokenizer to DataFrame
wordsData = tokenizer.transform(df)

# Apply hashing trick to words
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

# Compute TF-IDF features
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)



### Books recommendation system based on song listening situation

Define a UDF to apply the text_to_vector function to each row of the DataFrame

In [ ]:
from pyspark.ml.linalg import Vector, VectorUDT
# Define a function to convert a text string to a TF-IDF vector
def text_to_vector(text):
    words = text.split()
    wordsData = spark.createDataFrame([(0, words)], ["id", "words"])
    featurizedData = hashingTF.transform(wordsData)
    rescaledData = idfModel.transform(featurizedData)
    return rescaledData.select("features").first()[0]

# Define a UDF to apply the text_to_vector function to each row of the DataFrame
text_to_vector_udf = udf(text_to_vector, VectorUDT())

# Define the input text- song name
input_text = "I am listening to a love song"

# Convert the input text to a TF-IDF vector
input_vector = text_to_vector(input_text)




###Calculate cosine similarity between the input vector and the DataFrame vectors

In [ ]:

cosine_similarity_udf = udf(lambda x: float(input_vector.dot(x) / (input_vector.norm(2) * x.norm(2))), StringType())


###Get the top 10 book recommendation

In [ ]:
from pyspark.sql.types import StringType,BooleanType,DateType,DoubleType
final_df=rescaledData.select("title", cosine_similarity_udf("features").alias("cosine_similarity"))
# final_df.show()
final_df = final_df.withColumn("cosine_similarity", final_df["cosine_similarity"].cast(DoubleType()))
final_df.orderBy(col("cosine_similarity").desc()).limit(10).show(truncate=False)





+--------------------------------+------------------+
title |cosine_similarity |
+--------------------------------+------------------+
Poems That Touch the Heart |0.7117336561384187|
Being a Green Mother |0.6897578121430935|
What a Song Can Do |0.6620589650763575|
The "I Love Lucy" Book |0.6590175734881594|
Castle Roogna |0.6400154036549401|
J.M.W. Turner |0.6207636169425096|
In the Belly of the Beast |0.6186008639596937|
Phenomenal Woman |0.6175659527893639|
The Truelove Bride |0.6127886988806708|
Diamond Heart: The Freedom to Be|0.6112445940978041|
+--------------------------------+------------------+

###Conclusion:
Based on our analysis, we found that our recommendation system was able to accurately identify relevant books based on a user's current song listening situation. For example, when the user input "I am listening to a love song", our system recommended books such as "Poems That Touch the Heart" and "Being a Green Mother", which have a high degree of similarity to the user's input based on our model.

### Song recommendation system based on one book title
## Analytics Goal
The goal of this project is to recommend top 10 songs based on user input with a book title. To achieve this, we apply the TF-IDF model to train the song lyrics and calculate the similarity between the user input and the song lyrics using cosine similarity.

#### Load song lyrics data from MongoDB and Create features/label

In [ ]:
database = 'test_db'
collection = 'BookToSong_Lyrics'
user_name = 'chuang86'
password = 'msds697'
address = 'msds697-cluster.sig8e.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [ ]:
song_df = spark.read.format("mongo").option("uri",connection_string).load()

In [ ]:
song_df.show()

+--------------------+--------------------+--------------------+
 Song_lyrics| Song_name| _id|
+--------------------+--------------------+--------------------+
My Ántonia (Chap....| Gaillardia|[640ae2cebe326ef4...|
First of Many Lyr...| First of Many|[640ae2cebe326ef4...|
Baby, I’m an Anar...| In Reform|[640ae2cebe326ef4...|
Bleu Nuit Lyrics
...| Bleu Nuit|[640ae2cebe326ef4...|
American Reject L...|Interlude Among t...|[640ae2cebe326ef4...|
Midnight Sun Lyri...| Midnight Sun|[640ae2cebe326ef4...|
TranslationsTürkç...| Unknown|[640ae2cebe326ef4...|
Vanilla Baby Lyri...| Vanilla Baby|[640ae2cebe326ef4...|
Heteroflexible Ly...|The End of All Ou...|[640ae2cebe326ef4...|
TranslationsTürkç...| Summer|[640ae2cebe326ef4...|
Pound Cake / Pari...| Hopes|[640ae2cebe326ef4...|
Keeping Secrets L...| Keeping Secrets|[640ae2cebe326ef4...|
We Dance Lyrics(I...|The Mercy of the ...|[640ae2cebe326ef4...|
Suspension Lyrics...| Suspension|[640ae2cebe326ef4...|
San Junipero II L...| Kintsugi|[640ae2cebe326ef4...|
A Mandrake’s Gest...| Saga Of Me|[640ae2cebe326ef4...|
Woodland Lyrics
R...| Woodland|[640ae2cebe326ef4...|
It Always Rains O...| Whisperkiss|[640ae2cebe326ef4...|
Blessings (Repris...| Chamomile|[640ae2cebe326ef4...|
TranslationsEngli...| Poesia|[640ae2cebe326ef4...|
+--------------------+--------------------+--------------------+
only showing top 20 rows

### Tokenization on text and compute the TF-IDF features

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
# Define tokenizer
tokenizer = Tokenizer(inputCol="Song_lyrics", outputCol="words")

# Apply tokenizer to DataFrame
wordsData = tokenizer.transform(song_df)

# Apply hashing trick to words
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

# Compute TF-IDF features
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)




In [ ]:
from pyspark.ml.linalg import Vector, VectorUDT
# Define a function to convert a text string to a TF-IDF vector
def text_to_vector(text):
    words = text.split()
    wordsData = spark.createDataFrame([(0, words)], ["id", "words"])
    featurizedData = hashingTF.transform(wordsData)
    rescaledData = idfModel.transform(featurizedData)
    return rescaledData.select("features").first()[0]

# Define a UDF to apply the text_to_vector function to each row of the DataFrame
text_to_vector_udf = udf(text_to_vector, VectorUDT())

# Define the input text
input_text = "The Truelove Bride"

# Convert the input text to a TF-IDF vector
input_vector = text_to_vector(input_text)


### Calculate cosine similarity between the input vector and the DataFrame vectors

In [ ]:
cosine_similarity_udf = udf(lambda x: float(input_vector.dot(x) / (input_vector.norm(2) * x.norm(2))), StringType())

###Get the top 10 song recommendation

In [ ]:
from pyspark.sql.types import StringType,BooleanType,DateType,DoubleType
final_df=rescaledData.select("Song_name", cosine_similarity_udf("features").alias("cosine_similarity"))
# final_df.show()
final_df = final_df.withColumn("cosine_similarity", final_df["cosine_similarity"].cast(DoubleType()))
final_df.orderBy(col("cosine_similarity").desc()).limit(10).show()



+--------------------+------------------+
 Song_name| cosine_similarity|
+--------------------+------------------+
 There Will Be Rain|0.7247226152427609|
 Authenticity|0.6474468691304124|
Portrait of a Lon...| 0.629552671299129|
 Epiphany|0.6267799756484367|
 Ala|0.6152825878746678|
 Equipoise|0.6052418908089235|
 Woolgathering|0.6046448663520931|
Clair de Lune, L. 32| 0.590186036367559|
 Summer Memories|0.5873170606848128|
 Behind Frozen Eyes|0.5796934731864298|
+--------------------+------------------+